Comienzo TP 3

-Predicción de género musical a partir de un dataset de música.

Impostamos las librerias utilizadas

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve 
from sklearn.metrics import RocCurveDisplay
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import RobustScaler

In [2]:
data = pd.read_csv('.\music_genre.csv',sep=',')

data.head(10)


,instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre
0,32894.0,Röyksopp,Röyksopp's Night Out,27.0,0.00468,0.652,-1.0,0.941,0.792000,A#,0.1150,-5.201,Minor,0.0748,100.889,4-Apr,0.759,Electronic
1,46652.0,Thievery Corporation,The Shining Path,31.0,0.01270,0.622,218293.0,0.890,0.950000,D,0.1240,-7.043,Minor,0.0300,115.00200000000001,4-Apr,0.531,Electronic
2,30097.0,Dillon Francis,Hurricane,28.0,0.00306,0.620,215613.0,0.755,0.011800,G#,0.5340,-4.617,Major,0.0345,127.994,4-Apr,0.333,Electronic
3,62177.0,Dubloadz,Nitro,34.0,0.02540,0.774,166875.0,0.700,0.002530,C#,0.1570,-4.498,Major,0.2390,128.014,4-Apr,0.270,Electronic
4,24907.0,What So Not,Divide & Conquer,32.0,0.00465,0.638,222369.0,0.587,0.909000,F#,0.1570,-6.266,Major,0.0413,145.036,4-Apr,0.323,Electronic
5,89064.0,Axel Boman,Hello,47.0,0.00523,0.755,519468.0,0.731,0.854000,D,0.2160,-10.517,Minor,0.0412,?,4-Apr,0.614,Electronic
6,43760.0,Jordan Comolli,Clash,46.0,0.02890,0.572,214408.0,0.803,0.000008,B,0.1060,-4.294,Major,0.3510,149.995,4-Apr,0.230,Electronic
7,30738.0,Hraach,Delirio,43.0,0.02970,0.809,416132.0,0.706,0.903000,G,0.0635,-9.339,Minor,0.0484,120.008,4-Apr,0.761,Electronic
8,84950.0,Kayzo,NEVER ALONE,39.0,0.00299,0.509,292800.0,0.921,0.000276,F,0.1780,-3.175,Minor,0.2680,149.94799999999998,4-Apr,0.273,Electronic
9,56950.0,Shlump,Lazer Beam,22.0,0.00934,0.578,204800.0,0.731,0.011200,A,0.1110,-7.091,Minor,0.1730,139.933,4-Apr,0.203,Electronic


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50005 entries, 0 to 50004
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   instance_id       50000 non-null  float64
 1   artist_name       50000 non-null  object 
 2   track_name        50000 non-null  object 
 3   popularity        50000 non-null  float64
 4   acousticness      50000 non-null  float64
 5   danceability      50000 non-null  float64
 6   duration_ms       50000 non-null  float64
 7   energy            50000 non-null  float64
 8   instrumentalness  50000 non-null  float64
 9   key               50000 non-null  object 
 10  liveness          50000 non-null  float64
 11  loudness          50000 non-null  float64
 12  mode              50000 non-null  object 
 13  speechiness       50000 non-null  float64
 14  tempo             50000 non-null  object 
 15  obtained_date     50000 non-null  object 
 16  valence           50000 non-null  float6

In [4]:
data.replace(to_replace='?',value=np.NaN,inplace=True)

In [5]:
data.isnull().sum()

instance_id            5
artist_name            5
track_name             5
popularity             5
acousticness           5
danceability           5
duration_ms            5
energy                 5
instrumentalness       5
key                    5
liveness               5
loudness               5
mode                   5
speechiness            5
tempo               4985
obtained_date          5
valence                5
music_genre            5
dtype: int64

In [6]:
data.dropna(inplace=True)

In [7]:
data['tempo']=data.tempo.astype(float)

In [8]:
data.music_genre.unique()

array(['Electronic', 'Anime', 'Jazz', 'Alternative', 'Country', 'Rap',
       'Blues', 'Rock', 'Classical', 'Hip-Hop'], dtype=object)

In [9]:
data.obtained_date.unique()

array(['4-Apr', '3-Apr', '5-Apr', '1-Apr'], dtype=object)

La fecha no ofrece ningun dato

In [10]:
data.drop(columns=['obtained_date'],inplace=True)

In [11]:
data.drop(columns=['instance_id','artist_name','track_name'],inplace=True)

In [12]:
categorical_columns=[col for col in data.columns if data[col].dtype == 'object']
numerical_columns=[col for col in data.columns if data[col].dtype  != 'object']

In [13]:
categorical_columns

['key', 'mode', 'music_genre']

In [14]:
numerical_columns

['popularity',
 'acousticness',
 'danceability',
 'duration_ms',
 'energy',
 'instrumentalness',
 'liveness',
 'loudness',
 'speechiness',
 'tempo',
 'valence']

Preparamos nuestro X e y

In [15]:
X = data.drop(['music_genre'], axis=1)
y = data['music_genre']

Train, test, split

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state = 1237)

Feature engineering

In [17]:
X_categorical_columns=[col for col in categorical_columns if col != 'music_genre']

In [18]:
encoder_categories = []

for col in X_categorical_columns:    
    col_categories = data[col].unique()
    encoder_categories.append(col_categories)

encoder_categories

[array(['A#', 'D', 'G#', 'C#', 'F#', 'B', 'G', 'F', 'A', 'C', 'E', 'D#'],
       dtype=object),
 array(['Minor', 'Major'], dtype=object)]

In [19]:
encoder = OneHotEncoder(categories = encoder_categories, sparse=False)

encoder = encoder.fit(X_train[X_categorical_columns])
    
X_train_encoded = encoder.transform(X_train[X_categorical_columns])
X_train_categorical = pd.DataFrame(X_train_encoded, columns = encoder.get_feature_names(X_categorical_columns))

X_test_encoded = encoder.transform(X_test[X_categorical_columns])
X_test_categorical = pd.DataFrame(X_test_encoded, columns = encoder.get_feature_names(X_categorical_columns))
X_test_categorical.head()

c:\Users\nicol\anaconda3\envs\dh_sandbox\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\Users\nicol\anaconda3\envs\dh_sandbox\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,key_A#,key_D,key_G#,key_C#,key_F#,key_B,key_G,key_F,key_A,key_C,key_E,key_D#,mode_Minor,mode_Major
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [20]:
scaler = RobustScaler()

X_train_scaled = scaler.fit_transform(X_train[numerical_columns])
X_train_numerical = pd.DataFrame(X_train_scaled, columns = numerical_columns)
#X_train_numerical.head()

X_test_scaled = scaler.transform(X_test[numerical_columns])
X_test_numerical = pd.DataFrame(X_test_scaled, columns = numerical_columns)
X_test_numerical.head()

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,-0.045455,0.955994,0.579592,-0.561110,-0.545692,-0.000930,-0.191216,-0.233534,-0.3792,-0.438273,0.662404
1,-0.772727,1.521244,-0.559184,0.737978,-1.498433,5.735822,-0.074324,-1.739023,0.0288,-0.282569,-0.511509
2,0.681818,-0.265156,-1.428571,1.032782,0.900783,0.101612,1.675676,0.762925,1.7776,1.059351,-0.767263
3,0.818182,0.007587,0.351020,-0.450274,0.553525,-0.000996,-0.006757,0.470255,-0.2784,0.155550,1.258312
4,-0.500000,0.144158,-0.546939,0.898136,-0.203655,0.040888,0.250000,-0.051700,-0.2880,0.635422,-0.181586


In [21]:
Y_train_concat = Y_train.reset_index(drop=True)
Y_test_concat = Y_test.reset_index(drop=True)

data_train = pd.concat([X_train_categorical, X_train_numerical, Y_train_concat], axis=1)

data_test = pd.concat([X_test_categorical, X_test_numerical, Y_test_concat], axis=1)

In [22]:
data_train.columns

Index(['key_A#', 'key_D', 'key_G#', 'key_C#', 'key_F#', 'key_B', 'key_G',
       'key_F', 'key_A', 'key_C', 'key_E', 'key_D#', 'mode_Minor',
       'mode_Major', 'popularity', 'acousticness', 'danceability',
       'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness',
       'speechiness', 'tempo', 'valence', 'music_genre'],
      dtype='object')

In [23]:
X_train_2=data_train.drop(columns='music_genre')
y_train_2=data_train['music_genre']
X_test_2=data_test.drop(columns='music_genre')
y_test_2=data_test['music_genre']

In [24]:
X_train_2.shape

(33765, 25)

In [25]:
y_train_2.shape

(33765,)

In [26]:
gnb = GaussianNB()
gnb.fit(X_train_2, y_train_2)

GaussianNB()

In [27]:
y_pred = gnb.predict(X_test_2)

y_pred

array(['Country', 'Classical', 'Alternative', ..., 'Classical', 'Hip-Hop',
       'Rock'], dtype='<U11')

Performance

In [28]:
# accuracy
accuracy_score(y_test_2, y_pred)

0.4103953798311861

In [29]:
conf_mat = confusion_matrix(y_test_2, y_pred)

In [30]:
conf_mat

array([[194,   2,  44,  12, 420,  46, 192,  68,  50, 131],
       [ 30, 309, 119, 224, 252, 141,   3,  39,   3,   2],
       [ 38,  58, 327,  47, 365,  98,  12, 117,   4,  30],
       [ 15,  25,  28, 966,  26,  20,   0,  33,   1,   4],
       [ 42,   7,  59,  14, 789,  10,  90,  35,  24,  70],
       [ 72,  59, 105,  24, 113, 467,  98, 134,  11,  20],
       [ 45,   0,   4,   0,  97,  14, 627,  28, 265,  62],
       [ 33,  37, 148, 187, 132, 161,  60, 341,   2,  18],
       [ 26,   0,   3,   1, 121,   5, 601,  20, 249, 111],
       [ 69,   1,  15,  26, 356,  19, 113,  48, 122, 350]], dtype=int64)